<a href="https://colab.research.google.com/github/laisab/Inteligencia-Computacional/blob/main/projeto_inteligencia_computacional_v11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets accelerate peft bitsandbytes trl evaluate chromadb sentence-transformers
!pip install git+https://github.com/gpassero/uol-redacoes-xml.git

In [ ]:
import torch
import nltk
nltk.download('punkt_tab')
import uol_redacoes_xml
import pandas as pd
import numpy as np
import chromadb
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from datasets import Dataset
from evaluate import load
from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
essays = uol_redacoes_xml.load()
data = []

for essay in essays:
  if essay.comments.find('<comments>') or essay.comments.find('</comments>'):
    essay.comments = essay.comments.replace('<comments>', '')
    essay.comments = essay.comments.replace('</comments>', '')

  data.append({
      'THEME': essay.prompt.title,
      'ESSAY': essay.text,
      'COMMENTS': essay.comments
  })

filename = 'Essays.csv'
df_essays = pd.DataFrame(data)
df_essays.drop_duplicates(subset='ESSAY', inplace=True)
df_essays.dropna(inplace=True)
df_essays.to_csv(filename, index=False)

In [ ]:
# Modelo de embedding otimizado para português
EMBEDDING_MODEL_ID = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
EMBEDDING_MODEL = SentenceTransformer(EMBEDDING_MODEL_ID)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
model_id = "amadeusai/AV-FI-Qwen2.5-0.5B-PT-BR-Instruct"

# Tokenização do modelo pré-treinado
tok = AutoTokenizer.from_pretrained(model_id)
# Questão de segurança para modelos criptografados
if tok.pad_token is None: tok.pad_token = tok.eos_token

# Carregamento do modelo
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    dtype="auto",
    trust_remote_code=True # Permite o uso de modelos com arquiteturas mais recentes que a biblioteca não conhece
)
model

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/778 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/246 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2

In [ ]:
lora_cfg = LoraConfig(
    r=8, # Maior capacidade de aprender detalhes complexos devido a um número maior de parâmetros
    lora_alpha=16, # Força do impacto do que o LoRA aprendeu sobre o conhecimento original do modelo (valor alto = novo treino se sobrepõe ao modelo)
    lora_dropout=0.05, # A cada passo do treino, 5% dos neurônios do LoRA são "desligados"
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
)

# Envolve o modelo base (Sabiá) com o LoRA
model = get_peft_model(model, lora_cfg)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
      

In [ ]:
# Configuração do ChromaDB: biblioteca para implementação de banco de dados vetorial
chroma_client = chromadb.Client()

In [ ]:
# Criação de uma coleção (semelhante a uma tabela em um BD relacional)
collection_name = "essays_enem_feedback_context"
try:
  collection = chroma_client.get_collection(name=collection_name)
  print(f"Coleção '{collection_name}' já existe. Usando a existente.")
except:
  collection = chroma_client.create_collection(name=collection_name)
  print(f"Coleção '{collection_name}' criada.")

Coleção 'essays_enem_feedback_context' criada.


In [ ]:
# Função para gerar embeddings e adicionar ao ChromaDB
def add_to_vector_db(documents: list[str], metadatas: list[dict] = None, ids: list[str] = None):
  if not documents:
    return

  print(f"Gerando embeddings para {len(documents)} documentos")
  embeddings = EMBEDDING_MODEL.encode(documents).tolist()

  if ids is None:
    # ChromaDB pode gerar IDs automaticamente
    ids = [f"doc_{i}" for i in range(len(documents))]
    print(f"IDs não fornecidos, gerando IDs sequenciais")

  if metadatas is None:
    metadatas = [{}] * len(documents)

  print(f"Adicionando {len(documents)} documentos ao ChromaDB")
  collection.add(
      embeddings=embeddings,
      documents=documents,
      metadatas=metadatas,
      ids=ids
  )
  print(f"{len(documents)} documentos adicionados à coleção '{collection_name}'")

In [ ]:
# Função para buscar informações relevantes
def search_vector_db(query_text: str, n_results: int = 3) -> list[str]:
  print(f"Buscando no ChromaDB por: '{query_text[:50]}...'")
  query_embedding = EMBEDDING_MODEL.encode(query_text).tolist()

  results = collection.query(
      query_embeddings=[query_embedding],
      n_results=n_results,
      include=['documents', 'metadatas']
  )

  relevant_docs = []
  if results and results.get("documents") and results['documents'][0]:
    for doc, meta in zip(results['documents'][0], results.get('metadatas', [{}])[0]):
      relevant_docs.append(f"Contexto relevante:\n{doc}\n")

  print(f"Encontrados {len(relevant_docs)} documentos relevantes.")
  return relevant_docs

In [ ]:
print("Preparando dados para o banco vetorial")
docs_to_index = df_essays['COMMENTS'].tolist()
ids_to_index = [f"essay_comment_{i}" for i in range(len(df_essays))]

if docs_to_index:
  # Create minimal, non-empty metadata dictionaries to satisfy ChromaDB's validation
  metadatas_to_add = [{
      "type": "essay_comment"
  }] * len(docs_to_index)
  add_to_vector_db(documents=docs_to_index, metadatas=metadatas_to_add, ids=ids_to_index)
else:
  print("Nenhum comentário encontrado para indexar. Banco de dados vetorial não será populado.")

Preparando dados para o banco vetorial
Gerando embeddings para 2162 documentos
Adicionando 2162 documentos ao ChromaDB
2162 documentos adicionados à coleção 'essays_enem_feedback_context'


In [ ]:
RAG_IN_FMT = """Você é um assistente especializado em dar feedback para redações do ENEM.
Utilize os seguintes contextos para auxiliar na geração do feedback, focando em gramática, coesão, coerência e argumentação.
Seja construtivo e objetivo.

Contexto relevante:
{context}

Tema: {tema}

Redação:
{redacao}

Feedback:
"""

In [ ]:
# Tamanho máximo da sequência
MAX_LEN = 512

# Convert pandas DataFrame to Hugging Face Dataset
ds = Dataset.from_pandas(df_essays)

def tok_ex(ex):
  # Formatação da parte de instrução do prompt
  prompt = RAG_IN_FMT.format(tema=ex['THEME'], redacao=ex['ESSAY'])
  # Converte o texto do prompt em tokens sem adição de marcadores de início e fim de frase
  tp = tok(prompt, add_special_tokens=False)
  # Converte o texto da resposta em tokens sem adição de marcadores de início e fim de frase
  tr = tok(ex['COMMENTS'] + tok.eos_token, add_special_tokens=False)
  # Concatenar tokens de prompt e de resposta
  ids = (tp['input_ids'] + tr['input_ids'])[:MAX_LEN]
  # Criação da máscara de atenção
  att = [1] * len(ids)
  # Criação dos labels (modelo não deve treinar no prompt, apenas na resposta)
  lab = ([-100]*len(tp["input_ids"]) + tr["input_ids"])[:MAX_LEN] # -100 no prompt: o cálculo de loss dessa parte é ignorado
  # Exemplo tokenizado com os IDs de entrada, máscara de atenção e labels
  return {"input_ids": ids, "attention_mask": att, "labels": lab}

train_ds = ds.map(tok_ex, remove_columns=ds.column_names)
train_ds

Map:   0%|          | 0/2162 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2162
})

In [ ]:
# batch: lista com exemplos do dataset
def collate(batch):
  # Função auxiliar para auxiliar na organização das redações
  def pad_list(lst, pad_val):
    # Maior texto
    maxlen = max(len(x) for x in lst)

    out = []
    # Com os textos menores, adiciona um padding ao final até que fiquem do tamanho do maior
    for x in lst:
      # Conversão para um tensor
      t = torch.tensor(x, dtype=torch.long)
      # Adição do padding
      if t.size(0) < maxlen:
        t = torch.nn.functional.pad(t, (0, maxlen - t.size(0)), value=pad_val)
      out.append(t)

    # Transforma a lista em uma matriz (tensor)
    return torch.stack(out)

  # Extrai os input IDs de todos os exemplos no batch
  ids = [b["input_ids"] for b in batch]
  # Extrai as máscaras de atenção de todos os exemplos no batch
  att = [b["attention_mask"] for b in batch]
  # Extrai os labels de todos os exemplos no batch
  lab = [b["labels"] for b in batch]

  # Texto (input_ids)
  # Preenche o espaço vazio com um token nulo para o modelo saber que aquilo não é uma palavra

  # Atenção (attention_mask)
  # 0 para ignorar, caso contrário, o modelo tentará ler um espaço em branco

  # Gabarito (labels)
  # -100 para ignorar, assim o modelo não é avaliado por acertar um "espaço em branco"
  return{
      "input_ids": pad_list(ids, tok.pad_token_id),
      "attention_mask": pad_list(att, 0),
      "labels": pad_list(lab, -100)
  }

In [ ]:
args = TrainingArguments(
    output_dir="./redacao",
    num_train_epochs=1, # Número de épocas = quantidade de vezes que o dataset é apresentado para o modelo
    per_device_train_batch_size=2, # Tamanho da lista (batch) por GPU
    gradient_accumulation_steps=4, # Batchs efetivos = 16
    learning_rate=2e-4, # Taxa de aprendizado = velocidade que o modelo aprende
    logging_steps=10, # Frequência com que o treinamento imprime os resultados
    save_steps=50,
    report_to="none", # Desativa o envio de relatórios para ferramentas externas
    bf16=True, # GPU otimizada para trabalhar com 16 bits, acelerando cálculos
    optim="paged_adamw_32bit", # Evita que o treino seja interrompido for falta de recursos (quando a memória chega no limite)
    group_by_length=True # Organiza os batch para que textos de tamanhos parecidos fiquem juntos
)

trainer = Trainer(model=model, args=args, train_dataset=train_ds, data_collator=collate)
trainer.model.config

The model is already on multiple devices. Skipping the move to device specified in `args`.


Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "bfloat16",
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention"
  ],
  "max_position_embeddings": 32768,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_

In [ ]:
trainer.train()

Step,Training Loss
10,2.333500
20,2.118400
30,2.207000
40,2.166400
50,2.227400
60,2.188200
70,2.091600
80,2.112700
90,2.000300
100,2.037700


TrainOutput(global_step=271, training_loss=2.036385425342405, metrics={'train_runtime': 1282.1417, 'train_samples_per_second': 1.686, 'train_steps_per_second': 0.211, 'total_flos': 2404356490206720.0, 'train_loss': 2.036385425342405, 'epoch': 1.0})

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import sent_tokenize, word_tokenize

def avaliar_geracao(texto_original, texto_gerado):
  referencia = [word_tokenize(texto_original, language='portuguese')]
  gerado = word_tokenize(texto_gerado, language='portuguese')

  score = sentence_bleu(referencia, gerado)
  return score

In [ ]:
def generate_with_rag(instr):
  # Busca por contexto relevante
  search_query = instr['ESSAY']
  relevant_contexts = search_vector_db(search_query, n_results=3) # 3 trechos mais relevantes

  # Fromata o prompt para o modelo
  context_str = "\n".join(relevant_contexts)
  prompt = RAG_IN_FMT.format(context=context_str, tema=instr['THEME'], redacao=instr['ESSAY'])

  # Tokeniza o prompt para o modelo
  x = tok(prompt, return_tensors="pt").to(model.device)

  # Gera a resposta
  with torch.no_grad():
    y = model.generate(
      **x,
      max_new_tokens=1024, # Ajuste conforme necessário
      do_sample=True, # Para RAG, True é geralmente melhor para gerar texto mais natural
      top_p=0.95, # Exemplo de Top-P
      temperature=0.7, # Ajuste para controlar a criatividade (0.7 é um bom ponto de partida)
      eos_token_id=tok.eos_token_id,
      pad_token_id=tok.pad_token_id,
      no_repeat_ngram_size=3
    )

    # Processa a saída
    input_len = x["input_ids"].shape[1]
    generated_tokens = y[0][input_len:]
    generated_text = tok.decode(generated_tokens, skip_special_tokens=True)

    # Pega o gabarito original para comparação (se disponível)
    original_text = instr.get('COMMENTS', '') # Usa .get para evitar erro se não houver comentários

    # Avaliação (BLEU não é ideal para feedback, mas vamos manter para comparação)
    score = 0
    if original_text:
      score = avaliar_geracao(original_text, generated_text)
      print(f">> BLEU Score (comparado ao comentário original): {score:.3f}")
    else:
      print(">> Nenhum comentário original disponível para comparação.")

    print("\n" + "="*40)
    print(f"TEMA: {instr['THEME']}")
    print("-" * 40)
    print("SAÍDA DO MODELO (Gerado):")
    print(generated_text)
    print("-" * 40)
    if original_text:
      print("GABARITO (Original - apenas para referência):")
      print(original_text)
    print("="*40)

    return generated_text, score

In [ ]:
instrucao_para_gerar = df_essays.iloc[0]
feedback_gerado, bleu_score_rag = generate_with_rag(instrucao_para_gerar)

Buscando no ChromaDB por: 'A liberdade de expressão, assim como o direito à i...'
Encontrados 3 documentos relevantes.
>> BLEU Score (comparado ao comentário original): 0.053

TEMA: Direitos em conflito: liberdade de expressão e intimidade
----------------------------------------
SAÍDA DO MODELO (Gerado):
### Comentário geral
Apesar dos erros crassos e de uma linguagem muito superficial, o texto tem alguns parágrafos que demonstram compreendê-lo, apesar do pequeno número de trechos que faltaram. O que se nota mais foi a tentativa do autor de explicar os fatos que levam à análise. Mas isso não seria suficiente para apresentar uma reflexão sobre a questão, pois ele só cita alguns fatos (por exemplo, as imprensas em ditadurados ou fascistas), sem fazer referências a eles.
### Aspectos pontuais
1) Primeiro parágrafo: a) Não está claro qual é o problema do autor. b) O uso da palavra “conflito” é redundante, pois poderia-se dizer que existe conflito entre liberdades. c) Se a liberdad de expr